# Vehicle Classification

## Evaluate your model


In [14]:
import os

from sklearn.metrics import accuracy_score, classification_report

from utils import utils
from models import resnet_50


## Experiment settings

Set here the two variables in the following way:

- **CONFIG_YML:** assign the path to the config.yml file used for the experiment you want to evaluate
- **WEIGHTS:** assign the path to the model weights (.h5 file) you want to evaluate


In [15]:
# Modify this
CONFIG_YML = "../experiments/exp_014_v2/config_exp_014_v2.yml"
WEIGHTS = "../experiments/exp_014_v2/model.34-2.4452.h5"

In [16]:
# (!) Don't touch this cell, will load the config file specified before
config = utils.load_config(CONFIG_YML)
config

{'seed': 123,
 'data': {'directory': '/home/app/src/data/car_ims_v2/train',
  'labels': 'inferred',
  'label_mode': 'categorical',
  'validation_split': 0.2,
  'image_size': [224, 224],
  'batch_size': 32},
 'model': {'weights': 'imagenet',
  'input_shape': [224, 224, 3],
  'classes': 196,
  'dropout_rate': 0.3,
  'data_aug_layer': {'random_flip': {'mode': 'horizontal'},
   'random_zoom': {'height_factor': 0.1, 'width_factor': 0.1},
   'random_contrast': {'factor': 0.1}}},
 'compile': {'optimizer': {'adam': {'learning_rate': 0.0001,
    'epsilon': 0.01,
    'amsgrad': True}},
  'loss': 'categorical_crossentropy',
  'metrics': ['accuracy']},
 'fit': {'epochs': 100,
  'callbacks': {'model_checkpoint': {'filepath': '/home/app/src/experiments/exp_014_v2/model.{epoch:02d}-{val_loss:.4f}.h5',
    'save_best_only': True},
   'early_stopping': {'monitor': 'val_loss',
    'patience': 3,
    'restore_best_weights': True},
   'tensor_board': {'log_dir': '/home/app/src/experiments/exp_014_v2/logs'

In [17]:
# (!) Don't touch this cell, will use the config file to infer the class names
#     and also to load the corresponding testing dataset.
#     If you get an error, you must review your data/code.

MODEL_CLASSES = utils.get_class_names(config)

if len(MODEL_CLASSES) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your data!"
    )

_dirname, _ = os.path.split(config['data']['directory'])
TEST_FOLDER = os.path.join(_dirname, 'test')

if not os.path.exists(TEST_FOLDER):
    raise ValueError("'test' folder not found!")
    
if len(os.listdir(TEST_FOLDER)) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your testing dataset!"
    )
    
if set(os.listdir(TEST_FOLDER)) != set(MODEL_CLASSES):
    raise ValueError(
        "The name of the subfolders inside your test set "
        "doesn't match with the model classes!"
    )

## Load your model

Use `resnet_50.create_model()` and remember to properly setup the model weights!

Assign the model to the variable `cnn_model`.


In [18]:
# Complete this
# TODO
cnn_model = resnet_50.create_model(WEIGHTS)

# It should print your model correctly
print(cnn_model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                             

## Get predictions from testing dataset

In [19]:
# (!) Don't touch this cell, will use the loaded model and
#     the function utils.predict_from_folder() to get 
#     model predictions and the corresponding true labels
#     so we can measure the accuracy

predictions, labels = utils.predict_from_folder(
    folder=TEST_FOLDER, 
    model=cnn_model, 
    input_size=config["data"]["image_size"], 
    class_names=MODEL_CLASSES,
)

if len(predictions) != len(labels):
    raise ValueError(
        "The lenght of predictions and labels lists doesn't match!"
    )

if not isinstance(predictions[0], str):
    raise ValueError(
        "Model predictions should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )

if not isinstance(labels[0], str):
    raise ValueError(
        "Ground true labels should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )


In [20]:
# (!) Don't touch this cell

print(classification_report(y_true=labels, y_pred=predictions))


                                                        precision    recall  f1-score   support

                            AM General Hummer SUV 2000       0.63      0.89      0.74        44
                             Acura Integra Type R 2001       0.76      0.77      0.76        44
                                   Acura RL Sedan 2012       0.38      0.59      0.46        32
                                   Acura TL Sedan 2012       0.67      0.88      0.76        43
                                  Acura TL Type-S 2008       0.90      0.86      0.88        42
                                  Acura TSX Sedan 2012       1.00      0.40      0.57        40
                              Acura ZDX Hatchback 2012       0.94      0.79      0.86        39
              Aston Martin V8 Vantage Convertible 2012       0.39      0.58      0.47        45
                    Aston Martin V8 Vantage Coupe 2012       0.71      0.61      0.66        41
                  Aston Martin Virage C

In [21]:
# (!) Don't touch this cell

acc = accuracy_score(y_true=labels, y_pred=predictions)

print(f"Your model accuracy is {acc:.4f}!")

if acc < .3:
    raise ValueError("Your model accuracy is too low :(\nYou can do it better! :)")


Your model accuracy is 0.7230!
